In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1) ),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 00:44:59,754 INFO: Initializing external client
2025-03-04 00:44:59,754 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 00:45:01,090 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Fetching data from 2025-02-03 05:44:59.754201+00:00 to 2025-03-04 04:44:59.754201+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.28s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 06:00:00+00:00,2,0
1,2025-02-03 07:00:00+00:00,2,0
2,2025-02-03 08:00:00+00:00,2,0
3,2025-02-03 09:00:00+00:00,2,0
4,2025-02-03 10:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 00:00:00+00:00,263,5
174441,2025-03-04 01:00:00+00:00,263,6
174442,2025-03-04 02:00:00+00:00,263,6
174443,2025-03-04 03:00:00+00:00,263,3


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 180468
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 180468
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 00:45:11,788 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 00:45:11,800 INFO: Initializing external client
2025-03-04 00:45:11,800 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 00:45:12,621 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Fetching data from 2025-02-03 05:45:11.788296+00:00 to 2025-03-04 04:45:11.788296+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.83s) 


In [11]:
current_date

Timestamp('2025-03-04 05:45:11.788296+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 06:00:00
1,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,3,2025-03-03 06:00:00
2,2,3,3,4,6,2,2,2,4,4,...,1,3,1,0,1,2,0,1,4,2025-03-03 06:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 06:00:00
4,0,4,2,2,1,2,4,2,2,4,...,0,1,0,0,0,1,1,1,7,2025-03-03 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,259,2025-03-03 06:00:00
247,1,2,2,1,0,0,5,2,4,3,...,0,0,0,0,0,0,0,1,260,2025-03-03 06:00:00
248,2,5,11,10,7,26,13,25,37,25,...,7,5,0,0,0,0,2,2,261,2025-03-03 06:00:00
249,52,136,149,139,98,76,87,70,71,102,...,7,9,5,1,1,2,9,16,262,2025-03-03 06:00:00


In [13]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 00:45:23,643 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 00:45:23,655 INFO: Initializing external client
2025-03-04 00:45:23,655 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 00:45:24,410 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


In [14]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [15]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,1.0
...,...,...
246,259,0.0
247,260,1.0
248,261,1.0
249,262,39.0


In [16]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
122,132,150.0
174,186,101.0
95,100,61.0
151,162,56.0
44,48,55.0
223,236,52.0
226,239,45.0
225,238,45.0
250,263,39.0
249,262,39.0
